In [1]:
import numpy as np
import random
import pickle, os
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import random
from torch.utils.data import TensorDataset, DataLoader, random_split
from torch.nn import Sigmoid

from datetime import datetime
import time
import sys
import os
sys.path.append(os.path.abspath("..")) # Adds the parent folder to sys.path
from utils import *
from Neural_Nets import *

## Some useful functions 

In [2]:
# Convert an integer to 4 binaries (bits)
def int_to_four_bins(n):
    return np.array(list(np.binary_repr(int(n), width=4))).astype(int)

# This function computes the average accuracy per binary
def compute_bitwise_accuracy(preds, targets):
    return (preds == targets).float().mean().item()

# This function computes the strict accuracy: it counts how many entire binary vectors are predicted exactly right
def compute_exact_match_accuracy(preds, targets):
    return torch.all(preds == targets, dim=1).float().mean().item()

## Main classes

In [ ]:
class FFNet(torch.nn.Module):
    """Simple class to implement a feed-forward neural network in PyTorch.
    Attributes:
        layers: list of torch.nn.Linear layers to be applied in forward pass.
        activation: activation function to be applied between layers.
    """
    
    def __init__(self, shape, activation=None):
        """Constructor for FFNet.
        Arguments:
            shape: list of ints describing network shape, including input & output size.
            activation: a torch.nn function specifying the network activation.
        """
        super(FFNet, self).__init__()
        self.shape = shape
        self.layers = []
        self.activation = activation 
        for ii in range(0,len(shape)-1):
            self.layers.append(torch.nn.Linear(shape[ii],shape[ii+1]))
        self.layers = torch.nn.ModuleList(self.layers)

    def forward(self, x):
        "Performs a forward pass on x, a numpy array of size (-1,shape[0])"
        for ii in range(len(self.layers)-1):
            x = self.layers[ii](x)
            if self.activation:
                x = self.activation(x)

        return self.layers[-1](x)

In [8]:
class Regression:

    def __init__(self, prob_features):
        """
        Constructor for Regression class.
        """
        self.prob_features = prob_features
        self.n_bin = 4
        self.num_train, self.num_test = 0, 0
        self.model, self.model_fn = None, None

    def construct_features(self, params):
        prob_features = self.prob_features
        feature_vec = np.array([])

        for feature in prob_features:
            if feature == "x0":
                x0 = params['x0']
                feature_vec = np.hstack((feature_vec, x0))
            elif feature == "xg":
                xg = params['xg'] 
                feature_vec = np.hstack((feature_vec, xg))
            elif feature == "obstacles":
                obstacles = params['obstacles']
                feature_vec = np.hstack((feature_vec, np.reshape(obstacles, (4*self.n_obs))))
            elif feature == "obstacles_map":
                continue
            else:
                print('Feature {} is unknown'.format(feature))
        return feature_vec

    def setup_data(self, n_features, train_data):
        """
        Reads in data and constructs strategy dictionary
        """
        self.n_features = n_features

        self.X_train = train_data[0]
        self.Y_train = train_data[3]
        self.n_y = self.Y_train[0].size # will be the dimension of the output
        self.y_shape = self.Y_train[0].shape
        self.num_train = self.Y_train.shape[0]        

        self.features = np.zeros((self.num_train, self.n_features))
        self.labels = np.zeros((self.num_train, self.n_y))
        self.outputs = np.zeros((self.num_train, self.n_y*self.n_bin))
        
        for ii in range(self.num_train):
            self.labels[ii] = np.reshape(self.Y_train[ii,:,:], (self.n_y))
            self.outputs[ii] = np.hstack([int_to_four_bins(val) for val in (self.labels[ii])])
            prob_params = {}
            for k in self.X_train:
                prob_params[k] = self.X_train[k][ii]
            self.features[ii] = self.construct_features(prob_params)

    def setup_network(self, depth=3, neurons=32, device_id=0):
        self.device = torch.device('cuda:{}'.format(device_id))
        
        ff_shape = [self.n_features]
        for ii in range(depth):
            ff_shape.append(neurons)
        ff_shape.append(self.n_y*self.n_bin)

        self.model = FFNet(ff_shape, activation=torch.nn.ReLU()).to(device=self.device)

        # file names for PyTorch models
        now = datetime.now().strftime('%Y%m%d_%H%M')
        model_fn = 'regression_{}.pt'
        model_fn = os.path.join(os.getcwd(), model_fn)
        self.model_fn = model_fn.format(now)

    def load_network(self, fn_regressor_model):
        if os.path.exists(fn_regressor_model):
            print('Loading presaved regression model from {}'.format(fn_regressor_model))
            self.model.load_state_dict(torch.load(fn_regressor_model))
            self.model_fn = fn_regressor_model

    def train(self, training_params, verbose=True):
        # Unpack training params
        BATCH_SIZE = training_params['BATCH_SIZE']
        TEST_BATCH_SIZE = training_params['TEST_BATCH_SIZE']
        TRAINING_EPOCHS = training_params['TRAINING_EPOCHS']
        CHECKPOINT_AFTER = training_params['CHECKPOINT_AFTER']
        SAVEPOINT_AFTER = training_params['SAVEPOINT_AFTER']
        LEARNING_RATE = training_params['LEARNING_RATE']
        WEIGHT_DECAY = training_params['WEIGHT_DECAY']
        EARLY_STOPPING_PATIENCE = training_params['EARLY_STOPPING_PATIENCE']

        model = self.model
        device = self.device

        # Prepare dataset
        X_tensor = torch.from_numpy(self.features).float()
        Y_tensor = torch.from_numpy(self.outputs).float()
        full_dataset = TensorDataset(X_tensor, Y_tensor)

        # Split into train/val
        num_total = len(full_dataset)
        num_train = int(0.9 * num_total)
        train_dataset = TensorDataset(X_tensor[:num_train], Y_tensor[:num_train])
        val_dataset   = TensorDataset(X_tensor[num_train:], Y_tensor[num_train:])
        train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=TEST_BATCH_SIZE, shuffle=False)

        # Loss and optimizer
        loss_fn = nn.BCEWithLogitsLoss()
        optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
        
        best_val_loss = float('inf')
        epochs_since_improvement = 0

        itr = 1
        for epoch in range(TRAINING_EPOCHS):
            model.train()
            for inputs, y_true in train_loader:
                inputs = inputs.to(device)
                y_true = y_true.to(device)

                optimizer.zero_grad()
                logits = model(inputs)
                loss = loss_fn(logits, y_true)
                loss.backward()
                optimizer.step()

                if itr % CHECKPOINT_AFTER == 0:
                    # Evaluate on validation set
                    model.eval()
                    with torch.no_grad():
                        val_loss_total = 0
                        bitwise_accs = []
                        exact_match_accs = []

                        for val_inputs, val_targets in val_loader:
                            val_inputs = val_inputs.to(device)
                            val_targets = val_targets.to(device)

                            val_logits = model(val_inputs)
                            val_loss = loss_fn(val_logits, val_targets)
                            val_loss_total += val_loss.item()

                            val_probs = Sigmoid()(val_logits)
                            val_preds = (val_probs > 0.5).int()
                            val_targets_int = val_targets.int()

                            bitwise_accs.append(compute_bitwise_accuracy(val_preds, val_targets_int))
                            exact_match_accs.append(compute_exact_match_accuracy(val_preds, val_targets_int))

                        avg_val_loss = val_loss_total / len(val_loader)
                        avg_bitwise_acc = np.mean(bitwise_accs)
                        # avg_exact_acc = np.mean(exact_match_accs)

                        if verbose:
                            print(f"[Iter {itr}] Validation loss: {avg_val_loss:.4f} | "
                                f"Validation accuracy (bitwise): {avg_bitwise_acc:.4f}")
                            
                        # Check for early stopping
                        if avg_val_loss < best_val_loss - 1e-3:
                            best_val_loss = avg_val_loss
                            epochs_since_improvement = 0
                        else:
                            epochs_since_improvement += 1
                            if epochs_since_improvement >= EARLY_STOPPING_PATIENCE:
                                print(f"Early stopping: no improvement for {EARLY_STOPPING_PATIENCE} checks")
                                torch.save(model.state_dict(), self.model_fn)
                                print(f"Final model saved at {self.model_fn}")
                                return  # Exit training early

                    model.train()

                if itr % SAVEPOINT_AFTER == 0:
                    torch.save(model.state_dict(), self.model_fn)
                    if verbose:
                        print(f"[Iter {itr}] Saved model at {self.model_fn}")

                itr += 1

        # Save final model
        torch.save(model.state_dict(), self.model_fn)
        print(f"Final model saved at {self.model_fn}")
        print("Done training.")

In [9]:
relative_path = os.getcwd()
relative_path = os.path.abspath("..")
dataset_fn = relative_path + '/data' + '/single.p'
prob_features = ['x0', 'xg']

data_file = open(dataset_fn,'rb')
all_data = pickle.load(data_file)
data_file.close()
data_list = []
num_train = len(all_data)

X0 = np.vstack([all_data[ii]['x0'].T for ii in range(num_train)])  
XG = np.vstack([all_data[ii]['xg'].T for ii in range(num_train)])  
OBS = np.vstack([all_data[ii]['xg'].T for ii in range(num_train)])  
YY = np.concatenate([all_data[ii]['YY'].astype(int) for ii in range(num_train)], axis=1).transpose(1,0,2)

train_data = [{'x0': X0, 'xg': XG}, None, None, YY, None]

# Regression with Feed-forward NNs

In [10]:
# Build the FFNet model
FFNet_reg = Regression(prob_features)
n_features = 6 # the dimension of feature (input vector)
FFNet_reg.setup_data(n_features, train_data)

In [12]:
training_params = {}
training_params['TRAINING_EPOCHS'] = int(2000)
training_params['BATCH_SIZE'] = 100
training_params['CHECKPOINT_AFTER'] = int(1e3)
training_params['SAVEPOINT_AFTER'] = int(1e4)
training_params['TEST_BATCH_SIZE'] = 100
training_params['LEARNING_RATE'] = 1e-3
training_params['WEIGHT_DECAY'] = 1e-4
training_params['EARLY_STOPPING_PATIENCE'] = 10
FFNet_reg.setup_network(depth=4, neurons=1024)
FFNet_reg.model
FFNet_reg.train(training_params, verbose=True)

[Iter 1000] val loss: 0.0576 | bitwise acc: 0.9736
[Iter 2000] val loss: 0.0411 | bitwise acc: 0.9824
[Iter 3000] val loss: 0.0387 | bitwise acc: 0.9840
[Iter 4000] val loss: 0.0390 | bitwise acc: 0.9828
[Iter 5000] val loss: 0.0348 | bitwise acc: 0.9854
[Iter 6000] val loss: 0.0375 | bitwise acc: 0.9835
[Iter 7000] val loss: 0.0333 | bitwise acc: 0.9856
[Iter 8000] val loss: 0.0315 | bitwise acc: 0.9869
[Iter 9000] val loss: 0.0323 | bitwise acc: 0.9866
[Iter 10000] val loss: 0.0415 | bitwise acc: 0.9817
[Iter 10000] Saved model at /home/vietanhle/github/Python/SS_Learning_MIQP/self_supervised/regression_20250716_1515.pt
[Iter 11000] val loss: 0.0346 | bitwise acc: 0.9852
[Iter 12000] val loss: 0.0316 | bitwise acc: 0.9865
[Iter 13000] val loss: 0.0295 | bitwise acc: 0.9877
[Iter 14000] val loss: 0.0296 | bitwise acc: 0.9877
[Iter 15000] val loss: 0.0319 | bitwise acc: 0.9864
[Iter 16000] val loss: 0.0353 | bitwise acc: 0.9844
[Iter 17000] val loss: 0.0280 | bitwise acc: 0.9886
[Iter 